In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/NLP Project/Random Forest

/content/drive/.shortcut-targets-by-id/1owyyAUs7lGGypmp5pg74v7uwptu-YPm9/NLP Project/Random Forest


In [3]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
!pip install simpletransformers
!pip install h5py
import h5py
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import pandas as pd
import numpy as np
import random
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import model_selection as md
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from simpletransformers.language_representation import RepresentationModel
from sklearn.metrics import confusion_matrix
# import pandas as pd
import logging
from sklearn.metrics import precision_recall_fscore_support

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
seed = 50 #setting up random seed value
random.seed(seed)

In [5]:
df = pd.read_json('/content/drive/MyDrive/NLP Project/MUStARD/data/sarcasm_data.json').transpose() #reading utterances from json file
df['sarcasm']=df['sarcasm'].apply(lambda x: 1 if x==True else 0)
df['sarcasm'].value_counts()
df['index']=df.index.astype(str)
df['index']=df['index'].apply(lambda x: x[:1]+"_"+x[1:]) #changing ids for utterances in a form carried over in audio and video feature files
df=pd.get_dummies(df,columns=['speaker'])

In [6]:
def preprocessing(data): #preproceesing text
    data=data.lower()
    contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}
    for i in contractions: #replacing all contractions with their expanded forms
        data=data.replace(i,contractions[i])
    data=data.replace(r"[^?!,a-zA-Z0-9 ]","")
    
    return data

In [7]:
df["utterance"]=df["utterance"].apply(preprocessing) #applying preprocessing on utterances
df['context']=df['context'].apply(lambda x: [preprocessing(i) for i in x]) #applying preprocessing on context dialogues
df['context']=df['context'].apply(lambda x: ' '.join(x))

In [8]:
def getFeatures(x): #getting a fixed length audio features
  features = [0]*5094
  i = 0
  for x1 in x[0]:
    for x2 in x1:
      features[i] = x2
      i = i+1
  return features

In [9]:
def get_img(X): #generating flattened visual features of fixed size 265000 for all datapoints
  keys=list(X[:,5])
  img_embed=[]
  for i in keys:
    key=str(i)
    n1=hf.get(key)
    n1=np.array(n1).flatten() #flattening vectors
    if n1.shape[0]<265000:
      result = np.zeros(265000)
      result[:n1.shape[0]] = n1 #padding with 0 if size <265000
      n1=result
    else:
      n1=n1[:265000] #slicing to 265000 if size greater than 265000
    n1=list(n1)
    img_embed.append(n1)
  return np.array(img_embed)

In [10]:
def get_audio(X): #helps in audio feature generation of feature vectors
  keys=X[:,5]
  audio=[]
  for i in keys:
    audio.append(np.array(df_audio.loc[i]))
  return np.array(audio)

In [11]:
df_audio = pd.read_pickle('/content/drive/MyDrive/NLP Project/MUStARD/data/audio_features.p') #importing audio features in their raw forms
df_audio = pd.DataFrame.from_dict(df_audio, orient='index')
df_audio=df_audio.apply(getFeatures, axis=1, result_type='expand')
hf = h5py.File('/content/drive/MyDrive/NLP Project/MUStARD/data/features/utterances_final/resnet_pool5.hdf5', 'r') #importing visual features in their raw expanded forms
df['visual']=list(get_img(np.array(df)))
df['audio']=list(get_audio(np.array(df)))

In [12]:
def vaderScores(sentence): #generating vader sentiment scores
  obj = SentimentIntensityAnalyzer()
  v = obj.polarity_scores(sentence)
  sentiment = []
  sentiment.append(v['pos'])
  sentiment.append(v['neg'])
  sentiment.append(v['neu'])
  sentiment.append(v['compound'])

  return sentiment

In [13]:
X=df.drop(['sarcasm'],axis=1)
Y=df['sarcasm']
tfidf_vec_ut = TfidfVectorizer(ngram_range=(1, 2), max_features=500000, min_df=2)
tfidf_vec_ut.fit(X['utterance']) #generating tf-idf ngram embeddings for utterances
tfidf_vec_context = TfidfVectorizer(ngram_range=(1, 2), max_features=500000, min_df=2)
tfidf_vec_context.fit(X['context']) #generating tf-idf ngram embeddings for context

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0,
                max_features=500000, min_df=2, ngram_range=(1, 2), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words=None,
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [14]:
def get_sentiment(text): #fetching vader sentiment scores
  sentiment_scores=[]
  for i in text:
    sentiment_scores.append(vaderScores(i))
  return np.array(sentiment_scores)

In the following function, we continuously included or secluded code lines for inclusion and seclusion of different features. We have implemented the following features for various comparisons:  
* Text(T)
* Audio(A)
* Visual(V)
* T+V
* T+A
* A+V
* T+A+V

In [15]:
def create_feature_vec(X):
  tfidf_ut=tfidf_vec_ut.transform(X[:,0]) # tf-idf utterances embedding
  tfidf_context=tfidf_vec_context.transform(X[:,1]) #tf-idf context utterance embeddings
  speaker=np.array(X[:,5:X.shape[1]-2]) #speaker one hot encoding vector
  visual=np.array(X[:,X.shape[1]-2]) #visual flattened vector of length 265000
  audio=np.array(X[:,X.shape[1]-1]) #audio features using libroso
  sentiment_scores=get_sentiment(X[:,0]) #vader sentiment scores for utterances
  sentiment_scores_con=get_sentiment(X[:,1]) #vader sentiment scores for context utterances
  feat_vec=[np.concatenate((tfidf_ut[i].toarray()[0],tfidf_context.toarray()[0],speaker[i],visual[i],audio[i],sentiment_scores[i],sentiment_scores_con[i])) for i in range(X.shape[0])] #computing final vector
  return feat_vec

In [16]:
def cross_val_helper(clf,X,y,train,test):
  train_df = np.array(X)[train]
  test_df = np.array(X)[test]
  Y=np.array(y.astype(int))
  Y_train=Y[train]
  Y_test=Y[test]
  feat_vec=np.array(create_feature_vec(train_df))
  clf.fit(feat_vec,Y_train) #fitting our final model
  feat_vec_test=np.array(create_feature_vec(test_df))
  predictions = clf.predict(feat_vec_test) #predicting values using our final model
  CM= confusion_matrix(predictions, Y_test).ravel()
  print(CM)
  scores=precision_recall_fscore_support(Y_test,predictions, average='weighted')
  print("Precision: " + str(scores[0]))
  print("Recall: " + str(scores[1]))
  print("F1: "+ str(scores[2]))
  return [scores[0],scores[1],scores[2],clf]

In [17]:
n=3
kf = KFold(n_splits=n, random_state=seed, shuffle=True) #K-fold splits
train_kf=[]
test_kf=[]
for train, test in kf.split(X):
  train_kf.append(train)
  test_kf.append(test)

In [18]:
Precision_RFC=[]
Recall_RFC=[]
F1_RFC=[]
models=[]

We have shown only the implementing code for Random Forest but same function has been used for fitting and testing other models as well in a similar manner. The models implemented are:  
* RandomForestClassifier(max_depth=15,bootstrap=False,n_estimators=300)  
* XGBClassifier(max_depth=8)  
* SVC(C=13)  
* VotingClassifier(estimators=[('svc', SVC(C=13)), ('rf', RandomForestClassifier(max_depth=17,n_estimators=300)), ('xgb', XGBClassifier(max_depth=6,n_estimators=300))], voting='hard')

In [19]:
print("Random Forests") #we were forced to manually do 3-fold cross validation due to resource constraints
pr,rc,f1,model=cross_val_helper(RandomForestClassifier(max_depth=15,bootstrap=False,n_estimators=300),X,Y,train_kf[0],test_kf[0])
Precision_RFC.append(pr) #storing the precision
Recall_RFC.append(rc) #storing the recall
F1_RFC.append(f1) #storing the F1-score
models.append(model) #storing the model
outfile=open("RFC TAV Dep.pkl","wb") #saving the scores as well as the models
pickle.dump([Precision_RFC,Recall_RFC,F1_RFC,models],outfile)
outfile.close()

Random Forests
[69 28 44 89]
Precision: 0.6898898973116342
Recall: 0.6869565217391305
F1: 0.6850484472049688


In [20]:
print("Random Forests") #2nd fold
pr,rc,f1,model=cross_val_helper(RandomForestClassifier(max_depth=15,bootstrap=False,n_estimators=300),X,Y,train_kf[1],test_kf[1])
Precision_RFC.append(pr)
Recall_RFC.append(rc)
F1_RFC.append(f1)
models.append(model)
outfile=open("RFC TAV Dep.pkl","wb")
pickle.dump([Precision_RFC,Recall_RFC,F1_RFC,models],outfile)
outfile.close()

Random Forests
[69 22 50 89]
Precision: 0.7013161377252713
Recall: 0.6869565217391305
F1: 0.683617391304348


In [21]:
print("Random Forests") #3rd fold
pr,rc,f1,model=cross_val_helper(RandomForestClassifier(max_depth=15,bootstrap=False,n_estimators=300),X,Y,train_kf[2],test_kf[2])
Precision_RFC.append(pr)
Recall_RFC.append(rc)
F1_RFC.append(f1)
models.append(model)
outfile=open("RFC TAV Dep.pkl","wb")
pickle.dump([Precision_RFC,Recall_RFC,F1_RFC,models],outfile)
outfile.close()

Random Forests
[71 29 42 88]
Precision: 0.6931739130434783
Recall: 0.691304347826087
F1: 0.690007627765065


In [22]:
weighted_pr=sum(Precision_RFC)/3 #calculating average weighted scores
weighted_re=sum(Recall_RFC)/3
weighted_f1=sum(F1_RFC)/3

In [23]:
print("Random-Forest T+A+V Speaker Dependent")
print("Precision= ",weighted_pr)
print("Recall= ",weighted_re)
print("F1-score= ",weighted_f1)

Random-Forest T+A+V Speaker Dependent
Precision=  0.6947933160267946
Recall=  0.6884057971014492
F1-score=  0.6862244887581271
